In [67]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [68]:
path_archivos = "./Data/"
os.listdir(path_archivos)

['InversionComp.csv',
 'Sales.csv',
 'SpecialSale.csv',
 'InversionMedia.csv',
 'Sector.csv']

In [69]:
# Eccomerce CF
InversionMedia = pd.read_csv(path_archivos + "InversionMedia.csv")
Sales = pd.read_csv(path_archivos + "Sales.csv")
SpecialSale = pd.read_csv(path_archivos + "SpecialSale.csv")
# Total Sectores y competidores
InversionComp = pd.read_csv(path_archivos + "InversionComp.csv")
Sector = pd.read_csv(path_archivos + "Sector.csv")

In [70]:
Sales.shape

(104, 3)

## EDA

In [71]:
data_CF = pd.merge(InversionMedia, Sales, on="Date", how="left").rename(columns={"Pay TV": "PayTV", "Open TV": "OpenTV"})

In [72]:
data_CF.shape

(104, 11)

El Dataset cuenta con 12 variables/features:

- 1: tiempo - Date 
- 2: inversion - Print 
- 3: inversion - Other
- 4: inversion - Email
- 5: inversion - Radio      
- 6: inversion - Facebook    
- 7: inversion - Google      
- 8: inversion - PayTV       
- 9: inversion - OpenTV      
- 10: Ventas de la ecommers - Sales       
- 11: Score de Recomendacion de la pagina - NPS         
- 12: Dias de Ventas Especiales - SpecialSale (contiene valores nulos correspondientes a los dias que no son ventas especiales)

In [73]:
data_CF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 0 to 103
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      104 non-null    object
 1   Print     104 non-null    int64 
 2   Other     104 non-null    int64 
 3   Email     104 non-null    int64 
 4   Radio     104 non-null    int64 
 5   Facebook  104 non-null    int64 
 6   Google    104 non-null    int64 
 7   PayTV     104 non-null    int64 
 8   OpenTV    104 non-null    int64 
 9   Sales     104 non-null    int64 
 10  NPS       104 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 9.8+ KB


In [74]:
#data_CF['SpecialSale'] = data_CF['SpecialSale'].fillna("None") #Primero hacer en onehot luego en el merge con el dataframe de SpecialSale

- La mayor inversion se llevo acabo en OpenTV, con una inversion de $89,000
- La menor inversion se llevo acabo en Facebook y Google, con una inversion de $1,000
- Google y Facebook tienen una distribucion de inversiones muy similar 
- De igual manera la inversion en medios off como PayTV, Radio, Open TV son muy similares

In [75]:
data_desc = data_CF.describe().T
data_desc['Coef_Var'] = (data_desc['std'] / data_desc['mean']).round(3)
data_desc.sort_values(by='Coef_Var', ascending=False, inplace=True)
data_desc

,count,mean,std,min,25%,50%,75%,max,Coef_Var
PayTV,104.000,9778.846,11701.760,0.000,1000.000,5000.000,15000.000,55000.000,1.197
Radio,104.000,480.769,557.051,0.000,0.000,0.000,1000.000,3000.000,1.159
OpenTV,104.000,19067.308,21754.707,0.000,0.000,10000.000,35000.000,89000.000,1.141
Print,104.000,1721.154,1748.546,0.000,0.000,1000.000,3000.000,6000.000,1.016
Other,104.000,1201.923,1152.286,0.000,0.000,1000.000,2000.000,5000.000,0.959
Sales,104.000,73039.952,48124.139,3680.000,44487.000,61091.000,92539.250,284575.000,0.659
Email,104.000,3509.615,2207.644,0.000,2000.000,3000.000,5000.000,7000.000,0.629
Facebook,104.000,8961.538,4946.163,1000.000,5000.000,9000.000,12250.000,17000.000,0.552
Google,104.000,10865.385,5466.682,1000.000,6000.000,12000.000,15250.000,22000.000,0.503
NPS,104.000,60.269,16.307,26.000,48.500,62.500,72.000,95.000,0.271


In [87]:
px.histogram(data_CF, x=['Google','Facebook'],
            marginal="box",
            title="Histograma de " + 'Google y Facebook')

In [77]:
px.histogram(data_CF, x=['OpenTV','PayTV','Radio'],
            marginal="box",
            title="Histograma de " + 'Medios Off')

In [78]:
data_CF.columns

Index(['Date', 'Print', 'Other', 'Email', 'Radio', 'Facebook', 'Google',
       'PayTV', 'OpenTV', 'Sales', 'NPS'],
      dtype='object')

In [79]:
px.pie(pd.melt( data_CF, 
                id_vars=['Date'], 
                value_vars=['Print', 'Other', 'Email', 'Radio', 'Facebook', 'Google', 'PayTV', 'OpenTV'],
                var_name= "Medio",
                value_name= "Inversion"),
        values='Inversion', 
        names='Medio',
        title='Share of Investment by Media')

In [80]:
color_map = {'Print':'#1acfed', 'Other':'#fe6692', 'Email':'#ffa15a', 
            'Radio':'#b6e780', 'Facebook':'#ab63fa', 'Google':'#ef553b', 
            'PayTV':' #00cc96', 'OpenTV':'#636efa'}

In [81]:
data2 = data_CF.copy()
data2[['Print', 'Other', 'Email', 'Radio', 'Facebook', 'Google', 'PayTV', 'OpenTV']] = data2[['Print', 'Other', 'Email', 'Radio', 'Facebook', 'Google', 'PayTV', 'OpenTV']].apply(lambda x: x / x.sum(), axis=1)
px.bar( data2, 
        x ="Date", 
        y =['Print', 'Other', 'Email', 'Radio', 'Facebook', 'Google', 'PayTV', 'OpenTV'], 
        title ='Share of Investment by Media',
        color_discrete_map = color_map)

- La mayor parte de la inversion se realiza en TV aun que esta no esta tan activa en todo el periodo de tiempo
- Google y Facebook estan activos en la mayoria del tiempo

Al unir la base de special Sale se Duplican estos valores

In [82]:
data_CF.query("Date == '2021-02-14'")

,Date,Print,Other,Email,Radio,Facebook,Google,PayTV,OpenTV,Sales,NPS
58,2021-02-14,3000,2000,3000,1000,10000,14000,35000,55000,148582,63


In [83]:
data_CF.columns

Index(['Date', 'Print', 'Other', 'Email', 'Radio', 'Facebook', 'Google',
       'PayTV', 'OpenTV', 'Sales', 'NPS'],
      dtype='object')

In [84]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
medios = ['Print', 'Other', 'Email', 'Radio', 'Facebook', 'Google', 'PayTV', 'OpenTV']
for medio in medios:
    fig.add_trace(go.Bar(x=data_CF['Date'], y=data_CF[medio], name=medio), secondary_y=False)
fig.add_trace(go.Scatter(x=data_CF['Date'], y=data_CF['Sales'], name='Sales'), secondary_y=True)

fig.show()

In [85]:
import plotly.figure_factory as ff

df = data_CF.corr().round(2)

fig = ff.create_annotated_heatmap( df.values.tolist(),
                                    x=df.columns.values.tolist(),
                                    y=df.index.values.tolist(),
                                    colorscale='Viridis')
fig.update_layout(title_text='Correlacion')
fig['data'][0]['showscale'] = True
fig.show()

In [86]:
px.scatter_matrix(data_CF, dimensions=['Sales']+medios, height=800, width=800)